[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay21.ipynb)

# --- Day 21: Allergen Assessment ---

In [14]:
using System.IO;

In [15]:
record Food
{
    public List<string> Ingredients { get; init; }
    public List<string> Allergens { get; init; }
    
    public Food(string input)
    {
        Ingredients = input
            .Split(" (contains ")
            .First()
            .Split(" ")
            .ToList();
            
        Allergens = input
            .Split(" (contains ")
            .Last()
            .Replace(")", "")
            .Split(", ")
            .ToList();
    }
}

In [16]:
var foods = File.ReadAllLines(@"input/21.txt").Select(line => new Food(line));

In [17]:
var allergens = foods
    .Select(food => food.Allergens)
    .SelectMany(allergen => allergen)
    .Distinct()
    .ToDictionary(
        k => k,
        v => foods
            .Where(food => food.Allergens.Contains(v))
            .Select(food => food.Ingredients)
            .Aggregate<IEnumerable<string>>((a, b) => a.Intersect(b))
            .ToList()
    );
allergens

key,value
shellfish,"[ lrqqqsg, lcb ]"
wheat,"[ vhkj, lrqqqsg, shp, qzlmr ]"
peanuts,"[ vhkj, tvdvzd ]"
fish,[ vhkj ]
soy,"[ vhkj, dfzqlk ]"
dairy,"[ vhkj, smfz ]"
nuts,"[ qzlmr, tvdvzd ]"
sesame,"[ lcb, vhkj ]"


In [18]:
while (allergens.Where(kv => kv.Value.Count() > 1).Count() > 1)
{
    foreach (var allergen in allergens.Where(kv => kv.Value.Count() > 1))
    {
        allergen.Value.RemoveAll(ingredient => 
            allergens
                .Where(kv => kv.Value.Count() == 1)
                .Select(kv => kv.Value.First())
                .Contains(ingredient)
        );
    }
}
allergens

key,value
shellfish,[ lrqqqsg ]
wheat,[ shp ]
peanuts,[ tvdvzd ]
fish,[ vhkj ]
soy,[ dfzqlk ]
dairy,[ smfz ]
nuts,[ qzlmr ]
sesame,[ lcb ]


In [19]:
var allergenIngredients = allergens.Select(kv => kv.Value.First());
var nonAllergenIngredients = Enumerable.Except(
    foods.Select(food => food.Ingredients).SelectMany(f => f).Distinct(),
    allergenIngredients
);

In [20]:
nonAllergenIngredients
    .Select(i => foods
        .Select(food => food.Ingredients)
        .SelectMany(f => f)
        .Where(f => f == i).Count()
    )
    .Sum()

2302

# --- Part Two ---

In [21]:
string.Join(",", allergens.OrderBy(kv => kv.Key).Select(kv => kv.Value.First()))

smfz,vhkj,qzlmr,tvdvzd,lcb,lrqqqsg,dfzqlk,shp